In [1]:
class BTreeNode:
    def __init__(self, t, leaf=True):
        """
        Initialize a B-Tree node.

        Args:
            t (int): Minimum degree of the B-Tree.
            leaf (bool): True if the node is a leaf node.
        """
        self.t = t          # Minimum degree
        self.leaf = leaf    # Is this a leaf node?
        self.keys = []      # List of keys
        self.children = []  # List of child nodes

    def is_full(self):
        """Return True if the node has the maximum number of keys."""
        return len(self.keys) == 2 * self.t - 1

    def display(self, level=0):
        """
        Display the node and its children in a tree structure.
        
        Args:
            level (int): Current level in the tree (for indentation)
        """
        print(f"Level {level}: {self.keys}")
        if not self.leaf:
            for child in self.children:
                child.display(level + 1)

    def __str__(self):
        """Return a string representation of the node for debugging."""
        return f"Keys: {self.keys}, Leaf: {self.leaf}, Children: {len(self.children)}"


class BTree:
    def __init__(self, t=2):
        """
        Initialize a B-Tree with a given minimum degree.

        Args:
            t (int): Minimum degree (must be at least 2).
        """
        if t < 2:
            raise ValueError("Minimum degree must be at least 2")
        self.t = t
        self.root = BTreeNode(t, leaf=True)

    def search(self, k, node=None, level=0):
        """
        Search for a key in the B-Tree and return the node, index, and level.
        
        Args:
            k: Key to search for
            node: Current node (defaults to root)
            level: Current depth level (starts at 0 for root)
            
        Returns:
            Tuple (node, index, level) if found, None otherwise
        """
        if node is None:
            node = self.root
            
        i = 0
        while i < len(node.keys) and k > node.keys[i]:
            i += 1
            
        if i < len(node.keys) and k == node.keys[i]:
            return (node, i, level)
        elif node.leaf:
            return None
        else:
            return self.search(k, node.children[i], level + 1)
            
    def insert(self, k):
        """
        Insert a key into the B-Tree.

        Args:
            k: Key to insert.
        """
        root = self.root
        if root.is_full():
            new_root = BTreeNode(self.t, leaf=False)
            new_root.children.append(root)
            self.root = new_root
            self._split_child(new_root, 0)
            self._insert_non_full(new_root, k)
        else:
            self._insert_non_full(root, k)

    def _insert_non_full(self, node, k):
        """
        Insert key into a node that is not full.

        Args:
            node: Node to insert into.
            k: Key to insert.
        """
        i = len(node.keys) - 1
        if node.leaf:
            # Insert into leaf node
            node.keys.append(None)  # Create space for new key
            while i >= 0 and k < node.keys[i]:
                node.keys[i+1] = node.keys[i]
                i -= 1
            node.keys[i+1] = k
        else:
            # Find child where key should be inserted
            while i >= 0 and k < node.keys[i]:
                i -= 1
            i += 1
            
            # If child is full, split it
            if node.children[i].is_full():
                self._split_child(node, i)
                if k > node.keys[i]:
                    i += 1
            self._insert_non_full(node.children[i], k)

    def _split_child(self, parent, index):
        """
        Split a full child node into two.

        Args:
            parent: Parent node.
            index: Index of child to split.
        """
        t = self.t
        child = parent.children[index]
        
        # Create new node
        new_child = BTreeNode(t, leaf=child.leaf)
        
        # Move keys and children
        middle_key = child.keys[t-1]
        new_child.keys = child.keys[t:(2*t-1)]
        child.keys = child.keys[:(t-1)]
        
        if not child.leaf:
            new_child.children = child.children[t:(2*t)]
            child.children = child.children[:t]
        
        # Insert middle key and new child into parent
        parent.keys.insert(index, middle_key)
        parent.children.insert(index+1, new_child)

    def delete(self, k):
        """
        Delete key k from the B-Tree.
        
        Args:
            k: Key to delete
        """
        if not self.root.keys:
            return  # Empty tree
        
        self._delete_key(self.root, k)
        
        # If root has no keys and has a child, make the child the new root
        if not self.root.keys and not self.root.leaf:
            self.root = self.root.children[0]
    
    def _delete_key(self, node, k):
        """
        Delete key k from the given node.
        
        Args:
            node: Node to delete from
            k: Key to delete
        """
        t = self.t
        
        # Find the position of the key
        i = 0
        while i < len(node.keys) and k > node.keys[i]:
            i += 1
        
        # Case 1: Key is present in this node
        if i < len(node.keys) and node.keys[i] == k:
            if node.leaf:
                # Case 1: Simple removal from leaf
                node.keys.pop(i)
            else:
                # Case 2: Internal node
                # Case 2a: Left child has at least t keys
                if len(node.children[i].keys) >= t:
                    predecessor = self._get_predecessor(node.children[i])
                    node.keys[i] = predecessor
                    self._delete_key(node.children[i], predecessor)
                
                # Case 2b: Right child has at least t keys
                elif len(node.children[i+1].keys) >= t:
                    successor = self._get_successor(node.children[i+1])
                    node.keys[i] = successor
                    self._delete_key(node.children[i+1], successor)
                
                # Case 2c: Merge children
                else:
                    self._merge_children(node, i)
                    self._delete_key(node.children[i], k)
        else:
            # Case 3: Key is not in this node
            if node.leaf:
                return  # Key not found
            
            # Determine if key is in last child
            last_child = (i == len(node.keys))
            
            # Ensure child has at least t keys
            if len(node.children[i].keys) < t:
                self._fill_child(node, i)
                
                # After filling, adjust index if needed
                if last_child and i > len(node.keys):
                    i -= 1
            
            # Recursively delete from appropriate child
            self._delete_key(node.children[i], k)

    def _get_predecessor(self, node):
        """Get the predecessor key (maximum key in left subtree)."""
        while not node.leaf:
            node = node.children[-1]
        return node.keys[-1]

    def _get_successor(self, node):
        """Get the successor key (minimum key in right subtree)."""
        while not node.leaf:
            node = node.children[0]
        return node.keys[0]

    def _fill_child(self, node, index):
        """
        Fill a child that has t-1 keys by borrowing or merging.
        
        Args:
            node: Parent node
            index: Index of the child to fill
        """
        # Try to borrow from left sibling
        if index > 0 and len(node.children[index-1].keys) >= self.t:
            self._borrow_from_left(node, index)
        
        # Try to borrow from right sibling
        elif index < len(node.children)-1 and len(node.children[index+1].keys) >= self.t:
            self._borrow_from_right(node, index)
        
        # Merge with a sibling
        else:
            if index != len(node.children)-1:
                self._merge_children(node, index)
            else:
                self._merge_children(node, index-1)

    def _borrow_from_left(self, node, index):
        """
        Borrow a key from the left sibling.
        
        Args:
            node: Parent node
            index: Index of the child
        """
        child = node.children[index]
        left_sibling = node.children[index-1]
        
        # Move key from parent to child
        child.keys.insert(0, node.keys[index-1])
        
        # Move key from left sibling to parent
        node.keys[index-1] = left_sibling.keys.pop()
        
        # Move child pointer if necessary
        if not child.leaf:
            child.children.insert(0, left_sibling.children.pop())

    def _borrow_from_right(self, node, index):
        """
        Borrow a key from the right sibling.
        
        Args:
            node: Parent node
            index: Index of the child
        """
        child = node.children[index]
        right_sibling = node.children[index+1]
        
        # Move key from parent to child
        child.keys.append(node.keys[index])
        
        # Move key from right sibling to parent
        node.keys[index] = right_sibling.keys.pop(0)
        
        # Move child pointer if necessary
        if not child.leaf:
            child.children.append(right_sibling.children.pop(0))

    def _merge_children(self, node, index):
        """
        Merge node.children[index] with node.children[index+1].
        
        Args:
            node: Parent node
            index: Index of first child to merge
        """
        child = node.children[index]
        sibling = node.children[index+1]
        
        # Move key from parent to child
        child.keys.append(node.keys.pop(index))
        
        # Move keys from sibling
        child.keys.extend(sibling.keys)
        
        # Move children from sibling if not leaf
        if not child.leaf:
            child.children.extend(sibling.children)
        
        # Remove the sibling
        node.children.pop(index+1)
        
        # If root becomes empty, update root
        if node == self.root and not node.keys:
            self.root = child

    def display(self):
        """Display the B-Tree structure."""
        self.root.display()

    def inorder_traversal(self, node=None):
        """
        Perform an inorder traversal of the B-Tree.
        
        Args:
            node: Node to start from (defaults to root)
            
        Returns:
            List of keys in sorted order
        """
        if node is None:
            node = self.root
        
        result = []
        for i in range(len(node.keys)):
            if not node.leaf:
                result.extend(self.inorder_traversal(node.children[i]))
            result.append(node.keys[i])
        
        if not node.leaf:
            result.extend(self.inorder_traversal(node.children[-1]))
        
        return result

    def range_search(self, start, end):
        """
        Return all keys in the range [start, end].
        
        Args:
            start: Start of the range
            end: End of the range
            
        Returns:
            List of keys in the range
        """
        result = []
        self._range_search_helper(self.root, start, end, result)
        return result
    
    def _range_search_helper(self, node, start, end, result):
        """
        The helper for range_search that recursively finds all keys in the range [start, end].
        
        Args:
            node: Current node being processed
            start: Start of the range (inclusive)
            end: End of the range (inclusive)
            result: List to accumulate keys that fall within the range
        """
        i = 0
        while i < len(node.keys) and node.keys[i] < start:
            if not node.leaf:
                self._range_search_helper(node.children[i], start, end, result)
            i += 1

        while i < len(node.keys) and node.keys[i] <= end:
            if not node.leaf:
                self._range_search_helper(node.children[i], start, end, result)
            result.append(node.keys[i])
            i += 1

        if not node.leaf and i < len(node.children):
            self._range_search_helper(node.children[i], start, end, result)

In [2]:
# Create a B-Tree with minimum degree t = 3 (test)
btree = BTree(t=3)

# Insert nodes into the B-Tree
btree.insert(10)
btree.insert(20)
btree.insert(5)
btree.insert(6)
btree.insert(12)
btree.insert(30)
btree.insert(7)
btree.insert(17)

# Print the B-Tree after insertion
print("B-Tree after insertions:")
btree.display()
print("Inorder traversal:", btree.inorder_traversal())  # Inorder traversal will give sorted order of keys

B-Tree after insertions:
Level 0: [10]
Level 1: [5, 6, 7]
Level 1: [12, 17, 20, 30]
Inorder traversal: [5, 6, 7, 10, 12, 17, 20, 30]


In [3]:
# Search for a node
key_to_search = 12
result = btree.search(key_to_search)
if result:
    node, index, level = result
    print(f"Key {key_to_search} found:")
    print(f"- At level: {level}")
    print(f"- At index: {index}")
    print(f"- In node: {node}")
else:
    print(f"Key {key_to_search} not found")

Key 12 found:
- At level: 1
- At index: 0
- In node: Keys: [12, 17, 20, 30], Leaf: True, Children: 0


In [4]:
# Delete a node
key_to_delete = 6
print(f"Deleting key {key_to_delete} from the B-Tree...")
btree.delete(key_to_delete)

# Print the B-Tree after deletion
print("B-Tree after deletion:")
btree.display()
print("Inorder traversal:", btree.inorder_traversal())  # Inorder traversal will give sorted order of keys

Deleting key 6 from the B-Tree...
B-Tree after deletion:
Level 0: [10]
Level 1: [5, 7]
Level 1: [12, 17, 20, 30]
Inorder traversal: [5, 7, 10, 12, 17, 20, 30]


In [5]:
# Verify deletion
result = btree.search(key_to_delete)
print(f"Key {key_to_delete} found after deletion? {'Yes' if result else 'No'}")

# Range search
print("Range search [5, 17]:", btree.range_search(5, 17))

Key 6 found after deletion? No
Range search [5, 17]: [5, 7, 10, 12, 17]
